In [17]:
from pulp import *
import numpy as np
from mpi_utils import *

In [30]:
class Mcpp_var:
     def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D


def load_instances(num: int):
  if num < 10:
    num = "0"+str(num)
  else:
    num = str(num)
  
  file = open(f"../input/inst{num}.dat", 'r')
  
  m = int(file.readline())
  n = int(file.readline())
  l = [int(x) for x in file.readline().split(" ") if x!= ""]
  s = [int(x) for x in file.readline().split(" ") if x!= ""]
  D = []
  for i in range(n+1):
      D.append([int(x) for x in file.readline().split(" ") if x!= "\n" if x!= ""])
  
  return Mcpp_var(m, n, l, s, D)   

In [20]:
mcpp = load_instances(6)

# modello CVRP


In [33]:
#  has a value of 1 if the arc from node to node is in the optimal route and is driven by vehicle k
X = [[[ LpVariable(name=f'X_{i}_{j}_{k}', lowBound=0, upBound=1, cat=LpBinary) for k in range(mcpp.m) ] for j in range(mcpp.n + 1) ] for i in range(mcpp.n + 1)]

# Create the distance variables for each courier
dist_courier = [LpVariable(name=f'dist_cour{i}', cat=LpInteger , lowBound = 0, upBound=500) for i in range(mcpp.m)]
maximum = LpVariable(name=f'maximum', lowBound = 0, upBound =500 , cat = LpInteger)
model3 = LpProblem(name=f'mcpp3', sense=LpMinimize)


In [34]:
#1. vehicle leaves node that it enters
for j in range(mcpp.n + 1):
    for k in range(mcpp.m):
        model3 += lpSum([X[i][j][k] for i in range(mcpp.n + 1)]) == lpSum([X[j][i][k] for i in range(mcpp.n + 1)])

#2 every node is entered once
for j in range (mcpp.n):
    model3 += lpSum([[X[i][j][k] for i in range(mcpp.n + 1)] for k in range(mcpp.m)]) == 1

#no travel from a node to itself
for i in range(mcpp.n + 1):
    for k in range(mcpp.m):
        model3 += X[i][i][k] == 0

 #3 every vehicles leaves the depot
for k in range(mcpp.m):
   model3+= lpSum(X[mcpp.n][j][k] for j in range(mcpp.n)) == 1

#4 load constraint
for k in range(mcpp.m):
    model3 += lpSum([mcpp.s[j]*X[i][j][k] for j in range( mcpp.n ) for i in range(mcpp.n + 1)]) <= mcpp.l[k]

#subtours ----> ! TO DELETE AND MODIFY WITH A LAZY CONSTRAINT
for i in range(mcpp.n):
    for j in range(mcpp.n):
        if i != j:
            model3 += lpSum(X[i][j][k] for k in range(mcpp.m)) + lpSum(X[j][i][k] for k in range(mcpp.m)) <= 1

# distance constraint
for k in range(mcpp.m): 
  model3 += dist_courier[k] == lpSum(mcpp.D[i][j] * X[i][j][k] for j in range(mcpp.n + 1) for i in range(mcpp.n + 1))

#7. Goal function. We want to minimize the max distance.
for i in range(mcpp.m):
    model3 += maximum >= dist_courier[i] 
    
model3 += maximum
   
model3.solve() 

1

In [27]:
# Retrieve the solution values if needed
solution_matrix = [[[int(X[i][j][k].varValue) for k in range(mcpp.m)] for j in range(mcpp.n + 1)] for i in range(mcpp.n + 1)]
dist_cour_mat = [int(dist_courier[k].varValue) for k in range(mcpp.m)]

print("Solution Matrix:")
for row in solution_matrix:
     print(row)

# print(" \n pck Matrix:")
# for row in pck_matr:
#     print(row)
print(" \n dist mcpp:")
for row in mcpp.D:
     print(row)

print(" \n dist Matrix:")
for row in dist_cour_mat:
    print(row)





Solution Matrix:
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0

# Metodi modello 1 e 2

In [ ]:
def And(model, a, b, name):
        """
        And(a,b)
        :param a: first parameter of And condition
        :param b: second parameter of And condition
        :param name: name of the And
        :return: 1 if a and b is true, false otherwise
        """
        delta = LpVariable(cat=LpInteger, name=name)
        model += delta <= a
        model += delta >= a + b - 1
        model += delta >= 0
        model += delta <= b
        return delta

def linear_prod(model, binary_var, countinuos_var, ub, name):
        """
        res = binary_var * countinuos_var
        :param binary_var: binary variable
        :param countinuos_var: countinuos variable
        :param ub: upper bound of the countinuos variable
        :param name: name of the product
        :return: the result of the product
        """
        res = LpVariable(cat=LpInteger, name=name)
        model += ub * binary_var >= res
        model += countinuos_var >= res
        model += countinuos_var - (1 - binary_var) * ub <= res
        model += res >= 0
        return res

M = 1000

def If(model, a, b, M, name):
    delta = LpVariable(cat=LpBinary, name=name)
    model += a - b <= M * delta
    model += b - a <= M * (1 - delta)
    return delta




# Nuovo modello

Per il nuovo modello ho pensato di sfruttare una maschera (pck) per capire il percorso dei pacchi dato che era la cosa che negli altri approcci risultava un po' più problematica (non potento usare Lpvariable per scorrere la matrice D).
pck contiene i percorsi - da quale pacco a quale pacco - e X i pacchi presi da ciascun corriere. 
Ho pensato che se ogni corriere parte esattamente una volta deposito da li oi il percorso può essere ricostruito a ritroso senza avere un vettore per gli ordini.



In [22]:
#couriers and packages matrix
X = [[ LpVariable(name=f'X_{i}_{k}', lowBound=0, upBound=1, cat=LpBinary) for k in range(mcpp.n) ] for i in range(mcpp.m) ]

#binary mask of packages' path 
pck = [[ LpVariable(name=f'pck_{i}_{k}', lowBound=0, upBound=1, cat=LpBinary) for k in range(mcpp.n + 1)] for i in range(mcpp.n + 1) ]

# Create the distance variables for each courier
dist_courier = [LpVariable(name=f'dist_cour{i}', cat=LpInteger , lowBound = 0, upBound=500) for i in range(mcpp.m)]

# Create the distance variables for each courier


maximum = LpVariable(name='maximum', lowBound=0, upBound=500, cat=LpInteger)



In [23]:
# Create the PuLP model
model2 = LpProblem(name='mcpp2', sense=LpMinimize)

#1. each package has to be choosen 
for j in range(mcpp.n):
    model2 += lpSum([X[i][j] for i in range(mcpp.m)]) == 1 

#2.  pck diagonal should be 0
for i in range(mcpp.n + 1):
    model2 += pck[i][i] == 0   

#3. each package should have a path assigned 
for i in range(mcpp.n): 
    model2 += lpSum(pck[i][j] for j in range(mcpp.n  + 1)) == 1

for j in range(mcpp.n): 
    model2 += lpSum(pck[i][j] for i in range(mcpp.n  + 1)) ==1


#4.a each courier should bring at least one package (maybe useless)
for i in range(mcpp.m):
    model2 += lpSum(X[i][j] for j in range(mcpp.n)) >= 1    
#4.b since every courier bring a package m path should start from depot 
model2 += lpSum(pck[mcpp.n][j] for j in range(mcpp.n  + 1)) == mcpp.m
#and m path should end in depot 
model2 += lpSum(pck[j][mcpp.n] for j in range(mcpp.n  + 1)) == mcpp.m

#5. load size constraint 
for i in range(mcpp.m):
    model2 += lpSum([mcpp.s[j]*X[i][j] for j in range(mcpp.n) ]) <= mcpp.l[i]
    
#6. no loop
for i in range(mcpp.n):
    for j in range(mcpp.n):
        model2 += pck[i][j] + pck[j][i] <= 1

#7. distance constraint

for i in range(mcpp.m):
  model2 += dist_courier[i] == lpSum([[And(model2, X[i][j], pck[mcpp.n][j], f'and_dist_dep{i}_{mcpp.n}_{j}') * mcpp.D[mcpp.n][j]
                                      for j in range(mcpp.n  )], 
                                      [lpSum([And(model2, X[i][j], pck[j][j2], f'and_dist_{i}_{j}_{j2}') * mcpp.D[j][j2]
                                      for j2 in range(mcpp.n +1 ) for j in range(mcpp.n )])
                                      ]]) 
 



#8. the route has to be coherent with the assignment of the packages to the courier.
for i in range(mcpp.m):
    for j in range(mcpp.n):
     for j2 in range(mcpp.n):
        if(j != j2):
            model2 += And(model2, pck[j][j2], X[i][j], f'coh{i}{j}{j2}') == And(model2, pck[j][j2], X[i][j2], f'coh2_{i}{j}{j2}')


#8a. also for the depot
for i in range(mcpp.m):
        model2 += lpSum(And(model2, X[i][j], pck[mcpp.n][j], f'and_dep_{i}{j}') for j in range(mcpp.n)) == 1
        model2 += lpSum(And(model2, X[i][j], pck[j][mcpp.n], f'and_dep2_{i}{j}') for j in range(mcpp.n)) == 1


#10. Goal function. We want to minimize the max distance.
for i in range(mcpp.m):
    model2 += maximum >= dist_courier[i] 
    
model2 += maximum
         

# Solve the model
model2.solve(pulp.PULP_CBC_CMD())

1

In [ ]:
status = LpStatus[model2.status]

if status == "Optimal":
    print("La soluzione è ottima.")
elif status == "Infeasible":
    print("Il problema è infeasible.")
elif status == "Unbounded":
    print("Il problema è illimitato.")
elif status == "Undefined":
    print("Il problema è indefinito.")
else:
    print("Stato sconosciuto.")


La soluzione è ottima.


In [24]:
# Retrieve the solution values if needed
solution_matrix = [[int(X[i][k].varValue) for k in range(mcpp.n)] for i in range(mcpp.m)]
pck_matr = [[int(pck[i][k].varValue) for k in range(mcpp.n + 1)] for i in range(mcpp.n + 1)]
dist_cour_mat = [int(dist_courier[k].varValue) for k in range(mcpp.m)]

print("Solution Matrix:")
for row in solution_matrix:
    print(row)

print(" \n pck Matrix:")
for row in pck_matr:
    print(row)
print(" \n dist mcpp:")
for row in mcpp.D:
    print(row)
print(" \n dist Matrix:")
for row in dist_cour_mat:
    print(row)





Solution Matrix:
[0, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
 
 pck Matrix:
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 1, 1, 1, 0]
 
 dist mcpp:
[0, 80, 131, 22, 41, 127, 87, 48, 113]
[60, 0, 85, 82, 101, 81, 53, 106, 57]
[141, 83, 0, 129, 182, 4, 64, 189, 28]
[22, 82, 129, 0, 55, 125, 65, 66, 101]
[41, 99, 172, 55, 0, 168, 118, 11, 154]
[137, 81, 4, 125, 178, 0, 60, 185, 32]
[77, 63, 64, 65, 118, 60, 0, 125, 36]
[48, 126, 179, 60, 58, 175, 125, 0, 161]
[113, 57, 28, 101, 154, 24, 36, 161, 0]
 
 dist Matrix:
310
322
72
114
56
56


# Modello Binario Vecchio

In [ ]:
# matrix m*n*n+1 where we have couriers, order and packages
X  = [[[LpVariable(name = f'X_{i}_{k}_{j}', lowBound = 0, upBound = 1, cat = LpBinary) 
        for j in range(mcpp.n+1)] for k in range(mcpp.n)] for i in range(mcpp.m)]

#distance made by each courier
dist_courier = [LpVariable(name = f'dist_cour{i}', cat = LpInteger,lowBound = 0, upBound = np.max(mcpp.D)*(mcpp.n-2)) 
                for i in range(mcpp.m)]

maximum = LpVariable(name=f'maximum', lowBound = 0, upBound =np.max(mcpp.D)*(mcpp.n-2) , cat = LpInteger)

model = LpProblem(name=f'mcpp1', sense=LpMinimize)

#1. One hot encoding 
for i in range(mcpp.m):
    for k in range(mcpp.n):
        model += lpSum([X[i][k][j]for j in range(mcpp.n+1)]) == 1

#2. Each element only once in the cube
for j in range(mcpp.n):
    model += lpSum([[X[i][k][j] for k in range(mcpp.n)] for i in range(mcpp.m)]) == 1

#3. Load size constraint ( migliora con LpAffineSumExpression)
for i in range(mcpp.m):
    model += lpSum([mcpp.s[j]*X[i][k][j] for j in range(mcpp.n) for k in range(mcpp.n)]) <= mcpp.l[i]
 
#4. Every courier must start.
model += lpSum(X[i][0][mcpp.n] for i in range(mcpp.m)) == 0

#5. Constraint that if I see a 0, all the following element for a courier must be 0.
for i in range(mcpp.m):
    for k in range(mcpp.n-1):
            model += X[i][k][mcpp.n] - X[i][k+1][mcpp.n] <= 0 

# 6. Distances traveled by each courier
for i in range(mcpp.m):
    model += dist_courier[i] == lpSum([[X[i][0][j] * mcpp.D[mcpp.n][j] for j in range(mcpp.n)],[
    lpDot([And(model,X[i][j-1][k1],X[i][j][k2],f"{i}carries{k1}to{k2}in{j}") 
               for j in range(1,mcpp.n) for k1 in range(mcpp.n+1) for k2 in range(mcpp.n+1) if k1!= k2]
              ,[mcpp.D[k1][k2] for j in range(mcpp.n-1) for k1 in range(mcpp.n+1) for k2 in range(mcpp.n+1) if k1 != k2])]])

#7. Goal function. We want to minimize the max distance.
for i in range(mcpp.m):
    model += maximum >= dist_courier[i] 
    
model += maximum
   
model.solve()


1

In [ ]:
counter = 0
counter_pack = 0
pack = []
position = []
position_counter = 0
matrix = []

for var in model.variables():
    if "X" in var.name:
        if counter <= mcpp.n * mcpp.m * (mcpp.n + 1) - 1:
            pack.append(var.varValue)
            counter_pack += 1
            if counter_pack == mcpp.n+1:
                position.append(np.argmax(pack))
                counter_pack = 0
                pack = []
                position_counter += 1 
            if position_counter == (mcpp.n ):
                matrix.append(position)
                position = []
                position_counter = 0
            counter += 1
        
for row in matrix:
    print(row)

for var in model.variables():
    if "dist_cour" in var.name: 
        print(var.name, var.varValue)


NameError: name 'model' is not defined